# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096872


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/32 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/32 [00:04<01:05,  2.18s/it]

Rendering models:   9%|▉         | 3/32 [00:07<01:09,  2.40s/it]

Rendering models:  12%|█▎        | 4/32 [00:07<00:50,  1.79s/it]

Rendering models:  16%|█▌        | 5/32 [00:07<00:36,  1.34s/it]

...

Rendering models:  84%|████████▍ | 27/32 [00:12<00:01,  3.92it/s]

Rendering models:  88%|████████▊ | 28/32 [00:12<00:00,  4.48it/s]

Rendering models:  91%|█████████ | 29/32 [00:12<00:00,  4.51it/s]

Rendering models:  94%|█████████▍| 30/32 [00:12<00:00,  3.79it/s]

orangeninja27                         0.000237
not-logged-in-808d8d7eb7071532ff8d    0.000461
Jaronec                               0.000451
jnarayanbvg                           0.000549
Jonas_Cross                           0.001417
awright5                              0.000905
framosuw                              0.000813
gracie_solveig                        0.026219
sn346808                              0.001030
not-logged-in-14d09f70263c1fe15cc3    0.000698
Frosst.matt                           0.004512
not-logged-in-2105317ef1ec85b34132    0.004658
not-logged-in-eb2b769b73426d9a0d49    0.000817
GiulioC                               0.004041
Dargaan                               0.002700
Honhona                               0.104327
ElisabethB                            0.000270
jessicacaissie                        0.000384
ggravendust                           0.000208
not-logged-in-bcf0a63d16e116a99e14    0.001008
not-logged-in-bcf0a63d16e116a99e14    0.000446
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())